In [1]:
# %load D:/BUDS_MID/__init__.py
"""
Created on Wed Feb 19 16:21:00 2020

@author: cmchico

usual imports
"""

import os
import pandas as pd
import numpy as np

pd.options.display.max_rows = 100
pd.set_option('display.float_format', '{:.0f}'.format)


import f_validate_g4s as ed

from time import time
from datetime import timedelta

import pickle

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
# %load D:/BUDS_MID/__initdask__.py
"""
Created on Wed Feb 19 16:24:03 2020

@author: cmchico
"""
import dask.bag as db
from dask.distributed import Client, progress

client = Client(processes = True,n_workers=12)
client

'\nCreated on Wed Feb 19 16:24:03 2020\n\n@author: cmchico\n'

Client Scheduler: tcp://127.0.0.1:62503 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 12 Cores: 12 Memory: 34.14 GB


In [3]:
path = "D:/BUDS_MID/OJT ATM/ATM Forecasting Data New/"

In [4]:
thresh = pd.read_pickle(path+'89 Threshold Recommendations.pkl')
thresh = thresh[['tid','thresh']].drop_duplicates()
thresh.tid = thresh.tid.astype(str)
thresh.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 459 entries, 0 to 160758
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   tid     459 non-null    object 
 1   thresh  459 non-null    float64
dtypes: float64(1), object(1)
memory usage: 10.8+ KB


In [5]:
test_dates = pd.date_range(start='01/01/2020',end='12/31/2020')

In [6]:
mapp = pd.read_pickle(path+'Actual/89 TID Br map.pkl')
#211 branches with just one ATM, can be treated as an ATM i/o Branch
mapp[mapp.TID!=mapp.TID_BR].BR_TIDCOUNT.value_counts()

2    256
1    211
3     42
4      8
Name: BR_TIDCOUNT, dtype: int64

In [7]:
df = pd.concat([pd.read_csv(path+'Prophet Forecast 2020/'+i, dtype={'ds':'str','pred_date':'str','tid':'str'})\
                .assign(file=i)
                for i in os.listdir(path+'Prophet Forecast 2020/')],ignore_index=True)
df['tid'] = df.file.str.split('_').str[2].str.replace(".csv",'')
# df['pred_date_D'] = pd.to_datetime(df.pred_date)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 167994 entries, 0 to 167993
Data columns (total 5 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   Unnamed: 0  167994 non-null  int64  
 1   ds          167994 non-null  object 
 2   yhat        167994 non-null  float64
 3   file        167994 non-null  object 
 4   tid         167994 non-null  object 
dtypes: float64(1), int64(1), object(3)
memory usage: 6.4+ MB


In [8]:
df['yhat0'] = df['yhat'] 
df['yhat1'] = df['yhat'].shift(-1)
df['yhat2'] = df['yhat'].shift(-2)
df['y'] = df['yhat0']
df['date_str'] = df.ds.astype(str)
actual = df.copy()

In [9]:
atms = df.tid.unique()

In [115]:
def par(atm,thresh,rel_amt):
    
    date_o = pd.to_datetime('01/01/2019')
    actual_atm = actual[(actual.tid== atm)].reset_index(drop=True).copy()

    g4s=None
    threshold = thresh/rel_amt
    sched_= ed.sched(actual_atm,rel_amt=rel_amt,pred=pred,r=r,reload_charge=reload_charge,
                      threshold=threshold, hor=hor,g4s=g4s)
    sched_['fix_rel']=rel_amt
    sched_['thresh'] = thresh
    sched_['thresh%'] = threshold
    ss=sched_.groupby('cog_preemptive') \
                        [['reload_flag','cashout_flag','thresh_breach_flag','necessary_flag',
                                             'civ_cost','reload_cost']].sum().reset_index()
    ss['tid']=atm
    ss['fix_rel'] = rel_amt
    ss['thresh'] = thresh
    ss['thresh%'] = threshold
    sched_.to_csv(path+'Prophet Test Thresh R 2020/_sched_'+atm+'_'+str(rel_amt)[:2]+'.csv')
    ss.to_csv(path+'Prophet Test Perf Thresh R 2020/_schedperf_'+ atm+'_'+str(rel_amt)[:2]+'.csv')
    
    return sched_,ss

In [11]:
rel_amt=3200000
pred = ['yhat0','yhat1','yhat2']
hor=len(pred)-1
r = 0.05/365
reload_charge=2000

In [12]:
atm=atms[0]
thresh_ = thresh[thresh.tid==atm].reset_index(drop=True).thresh[0]
par(atm,thresh_,6200000)

   start_bal  start_reload  total_bal  end_bal   yhat
0    6072035             0    6072035  5996138  75897
1    5996138             0    5996138  5702115 294023
2    5702115             0    5702115  5475147 226968
3    5475147           nan        nan      nan    nan
   start_bal  start_reload  total_bal  end_bal   yhat
0    5996138             0    5996138  5702115 294023
1    5702115             0    5702115  5475147 226968
2    5475147             0    5475147  5475147      0
3    5475147           nan        nan      nan    nan
   start_bal  start_reload  total_bal  end_bal   yhat
0    5702115             0    5702115  5475147 226968
1    5475147             0    5475147  5475147      0
2    5475147             0    5475147  5456303  18844
3    5456303           nan        nan      nan    nan
   start_bal  start_reload  total_bal  end_bal   yhat
0    5475147             0    5475147  5475147      0
1    5475147             0    5475147  5456303  18844
2    5456303             0  

   start_bal  start_reload  total_bal  end_bal    yhat
0    1720164             0    1720164  1254804  465360
1    1254804             0    1254804   990274  264530
2     990274             0     990274  -733829 1724103
3          0           nan        nan      nan     nan
   start_bal  start_reload  total_bal  end_bal    yhat
0    1254804             0    1254804   990274  264530
1     990274       6200000    6200000  4475897 1724103
2    4475897             0    4475897  4142809  333088
3    4142809           nan        nan      nan     nan
   start_bal  start_reload  total_bal  end_bal    yhat
0     990274       6200000    6200000  4475897 1724103
1    4475897             0    4475897  4142809  333088
2    4142809             0    4142809  3871505  271304
3    3871505           nan        nan      nan     nan
   start_bal  start_reload  total_bal  end_bal   yhat
0    4475897             0    4475897  4142809 333088
1    4142809             0    4142809  3871505 271304
2    3871505 

   start_bal  start_reload  total_bal  end_bal   yhat
0    2785750             0    2785750  2693193  92558
1    2693193             0    2693193  2545059 148134
2    2545059             0    2545059  2419332 125727
3    2419332           nan        nan      nan    nan
   start_bal  start_reload  total_bal  end_bal   yhat
0    2693193             0    2693193  2545059 148134
1    2545059             0    2545059  2419332 125727
2    2419332             0    2419332  2266388 152944
3    2266388           nan        nan      nan    nan
   start_bal  start_reload  total_bal  end_bal   yhat
0    2545059             0    2545059  2419332 125727
1    2419332             0    2419332  2266388 152944
2    2266388             0    2266388  2054137 212251
3    2054137           nan        nan      nan    nan
   start_bal  start_reload  total_bal  end_bal   yhat
0    2419332             0    2419332  2266388 152944
1    2266388             0    2266388  2054137 212251
2    2054137             0  

   start_bal  start_reload  total_bal  end_bal    yhat
0    1928109             0    1928109  1717273  210835
1    1717273             0    1717273  1394707  322567
2    1394707             0    1394707  -554793 1949500
3          0           nan        nan      nan     nan
   start_bal  start_reload  total_bal  end_bal    yhat
0    1717273             0    1717273  1394707  322567
1    1394707       6200000    6200000  4250500 1949500
2    4250500             0    4250500  3663287  587213
3    3663287           nan        nan      nan     nan
   start_bal  start_reload  total_bal  end_bal    yhat
0    1394707       6200000    6200000  4250500 1949500
1    4250500             0    4250500  3663287  587213
2    3663287             0    3663287  3456336  206951
3    3456336           nan        nan      nan     nan
   start_bal  start_reload  total_bal  end_bal   yhat
0    4250500             0    4250500  3663287 587213
1    3663287             0    3663287  3456336 206951
2    3456336 

3    3108355           nan        nan      nan    nan
   start_bal  start_reload  total_bal  end_bal   yhat
0    3357625             0    3357625  3108355 249270
1    3108355             0    3108355  3108355      0
2    3108355             0    3108355  3101321   7034
3    3101321           nan        nan      nan    nan
   start_bal  start_reload  total_bal  end_bal  yhat
0    3108355             0    3108355  3108355     0
1    3108355             0    3108355  3101321  7034
2    3101321             0    3101321  3019059 82262
3    3019059           nan        nan      nan   nan
   start_bal  start_reload  total_bal  end_bal   yhat
0    3108355             0    3108355  3101321   7034
1    3101321             0    3101321  3019059  82262
2    3019059             0    3019059  2902649 116410
3    2902649           nan        nan      nan    nan
   start_bal  start_reload  total_bal  end_bal   yhat
0    3101321             0    3101321  3019059  82262
1    3019059             0    301

...

3    3000876           nan        nan      nan    nan
   start_bal  start_reload  total_bal  end_bal   yhat
0    3280083             0    3280083  3156597 123485
1    3156597             0    3156597  3000876 155721
2    3000876             0    3000876  2860147 140729
3    2860147           nan        nan      nan    nan
   start_bal  start_reload  total_bal  end_bal   yhat
0    3156597             0    3156597  3000876 155721
1    3000876             0    3000876  2860147 140729
2    2860147             0    2860147  2674977 185170
3    2674977           nan        nan      nan    nan
   start_bal  start_reload  total_bal  end_bal   yhat
0    3000876             0    3000876  2860147 140729
1    2860147             0    2860147  2674977 185170
2    2674977             0    2674977  2432485 242493
3    2432485           nan        nan      nan    nan
   start_bal  start_reload  total_bal  end_bal   yhat
0    2860147             0    2860147  2674977 185170
1    2674977             0  

   start_bal  start_reload  total_bal  end_bal   yhat
0    2094592             0    2094592  2084837   9756
1    2084837             0    2084837  2040714  44122
2    2040714             0    2040714  1885032 155682
3    1885032           nan        nan      nan    nan
   start_bal  start_reload  total_bal  end_bal   yhat
0    2084837             0    2084837  2040714  44122
1    2040714             0    2040714  1885032 155682
2    1885032             0    1885032  1681602 203430
3    1681602           nan        nan      nan    nan
   start_bal  start_reload  total_bal  end_bal    yhat
0    2040714             0    2040714  1885032  155682
1    1885032             0    1885032  1681602  203430
2    1681602             0    1681602  -158995 1840597
3          0           nan        nan      nan     nan
   start_bal  start_reload  total_bal  end_bal    yhat
0    1885032             0    1885032  1681602  203430
1    1681602       6200000    6200000  4359403 1840597
2    4359403        

3    3750465           nan        nan      nan     nan
   start_bal  start_reload  total_bal  end_bal   yhat
0    4317583             0    4317583  3996883 320700
1    3996883             0    3996883  3750465 246418
2    3750465             0    3750465  3502663 247802
3    3502663           nan        nan      nan    nan
   start_bal  start_reload  total_bal  end_bal   yhat
0    3996883             0    3996883  3750465 246418
1    3750465             0    3750465  3502663 247802
2    3502663             0    3502663  3485413  17250
3    3485413           nan        nan      nan    nan
   start_bal  start_reload  total_bal  end_bal   yhat
0    3750465             0    3750465  3502663 247802
1    3502663             0    3502663  3485413  17250
2    3485413             0    3485413  3485413      0
3    3485413           nan        nan      nan    nan
   start_bal  start_reload  total_bal  end_bal   yhat
0    3502663             0    3502663  3485413  17250
1    3485413             0 

   start_bal  start_reload  total_bal  end_bal   yhat
0    3234212             0    3234212  3098547 135665
1    3098547           nan        nan      nan  59694
2          0           nan        nan      nan 444992
3          0           nan        nan      nan    nan


(       date_str      y  yhat0  yhat1  yhat2  start_bal  start_reload  \
 0    2020-01-01 127965 127965  75897 294023          0       6200000   
 1    2020-01-02  75897  75897 294023 226968    6072035             0   
 2    2020-01-03 294023 294023 226968      0    5996138             0   
 3    2020-01-04 226968 226968      0  18844    5702115             0   
 4    2020-01-05      0      0  18844 180010    5475147             0   
 ..          ...    ...    ...    ...    ...        ...           ...   
 361  2020-12-27  81519  81519  20182 132762    3696302             0   
 362  2020-12-28  20182  20182 132762 227627    3614783             0   
 363  2020-12-29 132762 132762 227627 135665    3594601             0   
 364  2020-12-30 227627 227627 135665  59694    3461839             0   
 365  2020-12-31 135665 135665  59694 444992    3234212             0   
 
      total_bal  end_bal  last_load_amt  ... cashout_flag  thresh_breach_flag  \
 0      6200000  6072035              0  

In [ ]:
dbatms = db.from_sequence(atms)
dbthresh = db.from_sequence([thresh[thresh.tid==atm].reset_index(drop=True).thresh[0] for atm in atms])
dbmaster = dbatms.map(par,dbthresh,rel_amt=3200000)

start=time()
df = dbmaster.compute()
print(timedelta(seconds=time()-start))

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\distributed\client.py", line 2332, in get
    direct=direct)
  File "C:\ProgramData\Anaconda3\lib\site-packages\distributed\client.py", line 1656, in gather
    asynchronous=asynchronous)
  File "C:\ProgramData\Anaconda3\lib\site-packages\distributed\client.py", line 676, in sync
    return sync(self.loop, func, *args, **kwargs)
  File "C:\ProgramData\Anaconda3\lib\site-packages\distributed\utils.py", line 275, in sync
    e.wait(10)
  File "C:\ProgramData\Anaconda3\lib\threading.py", line 552, in wait
    signaled = self._cond.wait(timeout)
  File "C:\ProgramData\Anaconda3\lib\threading.py", line 300, in wait
    gotit = waiter.acquire(True, timeout)
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3331, in run_code
    exec(code_obj, sel

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

distributed.process - WARNING - reaping stray process <Process(Process-12, started daemon)>
distributed.process - WARNING - reaping stray process <Process(Process-6, started daemon)>


In [13]:
dbatms = db.from_sequence(atms)
dbthresh = db.from_sequence([thresh[thresh.tid==atm].reset_index(drop=True).thresh[0] for atm in atms])
dbmaster = dbatms.map(par,dbthresh,rel_amt=5200000)

start=time()
df = dbmaster.compute()
print(timedelta(seconds=time()-start))

0:12:18.332390


In [14]:
dbatms = db.from_sequence(atms)
dbthresh = db.from_sequence([thresh[thresh.tid==atm].reset_index(drop=True).thresh[0] for atm in atms])
dbmaster = dbatms.map(par,dbthresh,rel_amt=6200000)

start=time()
df = dbmaster.compute()
print(timedelta(seconds=time()-start))

0:12:26.850775


In [15]:
dbatms = db.from_sequence(atms)
dbthresh = db.from_sequence([thresh[thresh.tid==atm].reset_index(drop=True).thresh[0] for atm in atms])
dbmaster = dbatms.map(par,dbthresh,rel_amt=4200000)

start=time()
df = dbmaster.compute()
print(timedelta(seconds=time()-start))

0:12:41.841790


In [104]:
df = pd.concat([pd.read_csv(path+'Prophet Test Perf Thresh R 2020/'+i).assign(file=i) for 
                            i in os.listdir(path + 'Prophet Test Perf Thresh R 2020/')])

In [105]:
df['tid'] = df.file.str.split("_").str[2]
df.tid = df.tid.astype(str)

In [106]:
perf = df.groupby(['tid','fix_rel'])[['civ_cost','reload_cost']].sum().reset_index()
perf['total_cost'] = perf.civ_cost+perf.reload_cost

In [107]:
best = perf.iloc[perf.groupby(['tid']).total_cost.idxmin().values].copy()
best['tid']= best.tid.astype(str)

In [108]:
best.to_csv(path+'90 2020 Perf Thresh R.csv')

In [117]:
df2 = pd.concat([pd.read_csv(path+'Prophet Test Perf Thresh R/OFFSITE OR ONE ATM BRANCH/'+i).assign(file=i) for 
                            i in os.listdir(path + 'Prophet Test Perf Thresh R/OFFSITE OR ONE ATM BRANCH/')])

In [118]:
perf2 = df2.groupby(['tid','fix_rel'])[['civ_cost','reload_cost']].sum().reset_index()
perf2['total_cost'] = perf2.civ_cost+perf.reload_cost
df2['tid'] = df2.file.str.split("_").str[2]
df2.tid = df2.tid.astype(str)
best_19 = perf2.iloc[perf2.groupby(['tid']).total_cost.idxmin().values].copy()
best_19['tid'] = best_19.tid.astype(str)

In [119]:
all_ = best_19.merge(best, on='tid', how='left',suffixes=(['_2019','_2020']))

In [120]:
all_['change'] = all_['fix_rel_2019'] - all_['fix_rel_2020']

In [125]:
all_.change.value_counts()

 0          316
 1000000     26
-1000000     22
-3000000     19
-2000000      5
 3000000      3
 2000000      2
Name: change, dtype: int64

In [126]:
+26+3+2

31

In [127]:
22+19+5

46